In [0]:
import numpy as np

In [0]:
# Detector range (used for generation and representations)
ymin = -2
ymax = +2

In [0]:
def generate_event(type):
  if type == 1: # signal
    averageN = 20
    ymean = 1
    ysigma = 3
    pTslope = 10
  elif type == 2: # background
    averageN = 30
    ymean = -1
    ysigma = 3
    pTslope = 12
  else:
    raise Exception
    
  n = np.random.poisson(averageN)
  particles = []
  for i in range(n):
    y = np.random.normal(ymean, ysigma)
    if not ymin < y < ymax:
      continue
    pT = np.random.exponential(pTslope)
    phi = np.random.uniform(0, 2*np.pi)
    particles.append((pT, y, phi))
  
  return particles

In [0]:
# representing event as list of first N particles

def representation1(event, N, sort=0):
  # N - how many particles to keep
  # sort - by which variable to sort (0=Pt, 1=y, 2=phi)
  sorted_event = sorted(event, key=lambda x: x[sort])
  extended = sorted_event + N*[(-100, -100, -100)] # extending in case event is shorter than N
  shortened = extended[:N]
  flattened = [item for sublist in shortened for item in sublist]
  return flattened

In [0]:
# representing event as list of first N moments

def moment(A, B, Na, Nb):
  if not len(A) == len(B):
    raise Exception
  
  if Na == 0 and Nb == 0:
    return 1.0
  if Na == 1 and Nb == 0:
    return np.mean(A)
  if Na == 0 and Nb == 1:
    return np.mean(B)
  
  meanA = np.mean(A)
  meanB = np.mean(B)
  s = 0.0
  for a, b in zip(A, B):
    s += (a - meanA)**Na * (b-meanB)**Nb  
  return s/len(A)


def representation2(event, N):
  # N - number of moments
  data = zip(*event)
  M = []
  calculated_moments = []
  for nA in range(N+1):
    for nB in range(N+1):
      if nA == 0 and nB == 0:
        continue
      for iA in range(len(data)):
        for iB in range(len(data)):
          if nA == 0:
            mm = iB + 10*nB
          elif nB == 0:
            mm = iA + 10*nA
          else:
            mm = iA + 10*iB + 100*nA + 10000*nB
          if mm in calculated_moments:
            continue
          calculated_moments.append(mm)
          A = data[iA]
          B = data[iB]
          m = moment(A, B, nA, nB)
          M.append(m)
  return M

In [0]:
# representing event as a 2D histogram

def representation3(event, Ny, Nphi):  
  # Ny, Nphi - number of bins
  pT, y, phi = zip(*event)
  yrange = (ymin, ymax)
  phirange = (0, 2*np.pi)
  r = (yrange, phirange)
  b = (Ny, Nphi)
  h = np.histogram2d(y, phi, bins=b, range=r, weights=pT)
  return list(h[0].flatten())

In [0]:
# Parameters chosen so that the lengths of vectors in all representation are equal
a = generate_event(1)
r1 = representation1(a, 14)
r2 = representation2(a, 2)
r3 = representation3(a, 7, 6)
print len(a), a
print len(r1), r1
print len(r2), r2
print len(r3), r3

10 [(17.287623128376083, 0.9177619960181417, 6.227468721093823), (12.622289580708959, -1.0054222531588506, 5.591622075943847), (1.271921367061857, -1.5757035974215619, 0.48563648148442323), (6.936432624835229, 1.143475670724774, 0.20836873827151248), (16.560649811366524, 1.3227129802834745, 4.88912806709209), (7.6151657817556115, 1.328660897539884, 6.24172056891231), (13.151469273336874, -0.16285169968487945, 2.9793231909452125), (1.9813000699096486, 1.8233666761326166, 1.6005821891613696), (22.829484039126143, 1.3400074163256728, 1.0622478516303866), (8.015831903285273, -1.9702020383719692, 2.041412846421608)]
42 [1.271921367061857, -1.5757035974215619, 0.48563648148442323, 1.9813000699096486, 1.8233666761326166, 1.6005821891613696, 6.936432624835229, 1.143475670724774, 0.20836873827151248, 7.6151657817556115, 1.328660897539884, 6.24172056891231, 8.015831903285273, -1.9702020383719692, 2.041412846421608, 12.622289580708959, -1.0054222531588506, 5.591622075943847, 13.151469273336874, -